In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

import re, os
import unicodedata
import json

import nltk
import nltk.sentiment
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from time import strftime

from wordcloud import WordCloud

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# make sure these imports are before ->
import sys
directory_path = "/Users/dragonzord/Documents/SchoolDocs/codeup-data-science/exercises/methodologies2/capstone/nlp-capstone/src"
sys.path.append(directory_path)

# <- these imports
import acquire
import prepare_jag

import spacy
pd.set_option('display.max_colwidth', None)



### Acquire data

We will be using the 1000 labeled notes in `test.csv`

In [3]:
df = prepare_jag.prep_train()

In [4]:
df.head(1)


,Unnamed: 0,id,case_num,pn_num,feature_num,feature_text,annotation,location,original,clean,stemmed,lemmatized,original_word_count
0,0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction,['dad with recent heart attcak'],['696 724'],"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with ""thyroid disease,"" dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms",hpi 17yo presents palpitations patient reports 3-4 months intermittent episodes heart beating/pounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 1-3 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes vision/hearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 3-4 drinks 3 nights / week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms,hpi 17yo present palpit patient report 3-4 month intermitt episod heart beating/pound chest 2 day ago dure soccer game episod thi time chest pressur felt go pass lose concious note patient endors abus adderal primarili studi 1-3 time per week befor recent soccer game took adderral night befor morn game deni short breath diaphoresi fever chill headach fatigu chang sleep chang vision/hear abdomin paun chang bowel urinari habit pmhx none rx use friend adderral fhx mom thyroid diseas dad recent heart attcak none immun date shx freshmen colleg endors 3-4 drink 3 night / week weekend deni tabacco endors tri marijuana sexual activ girlfriend x 1 year use condom,hpi 17yo present palpitation patient report 3-4 month intermittent episode heart beating/pounding chest 2 day ago soccer game episode time chest pressure felt going pas lose conciousness note patient endorses abusing adderall primarily study 1-3 time per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fever chill headache fatigue change sleep change vision/hearing abdominal paun change bowel urinary habit pmhx none rx us friend adderrall fhx mom thyroid disease dad recent heart attcak none immunization date shx freshman college endorses 3-4 drink 3 night / week weekend denies tabacco endorses trying marijuana sexually active girlfriend x 1 year us condom,142


---

## Modeling

### Split Data

- Split in to train / test (700/300)


### X/y split

- Let x be the TFID vectorization of the lemmatized word list
- y = case number

In [5]:
df.dtypes

Unnamed: 0              int64
id                     object
case_num                int64
pn_num                  int64
feature_num             int64
feature_text           object
annotation             object
location               object
original               object
clean                  object
stemmed                object
lemmatized             object
original_word_count     int64
dtype: object

HAving issues 

In [6]:
df.case_num=df.case_num.astype('category')

In [7]:
df.dtypes

Unnamed: 0                int64
id                       object
case_num               category
pn_num                    int64
feature_num               int64
feature_text             object
annotation               object
location                 object
original                 object
clean                    object
stemmed                  object
lemmatized               object
original_word_count       int64
dtype: object

Running into issue with number that isnt really a number for a target rename

In [8]:
############### Column Lambdas###############


def categorise(row):
    if row['case_num'] == 0:
        return 'case0'
    elif row['case_num'] == 1:
        return 'case1'
    elif row['case_num'] == 2:
        return 'case2'
    elif row['case_num'] == 3:
        return 'case3'
    elif row['case_num'] == 4:
        return 'case3'
    elif row['case_num'] == 5:
        return 'case5'
    elif row['case_num'] == 6:
        return 'case6'
    elif row['case_num'] == 7:
        return 'case7'
    elif row['case_num'] == 8:
        return 'case8'
    return 'case9'

In [9]:
df['target'] = df.apply(lambda row: categorise(row), axis=1)


In [10]:
df.head(1)

,Unnamed: 0,id,case_num,pn_num,feature_num,feature_text,annotation,location,original,clean,stemmed,lemmatized,original_word_count,target
0,0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction,['dad with recent heart attcak'],['696 724'],"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with ""thyroid disease,"" dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms",hpi 17yo presents palpitations patient reports 3-4 months intermittent episodes heart beating/pounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 1-3 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes vision/hearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 3-4 drinks 3 nights / week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms,hpi 17yo present palpit patient report 3-4 month intermitt episod heart beating/pound chest 2 day ago dure soccer game episod thi time chest pressur felt go pass lose concious note patient endors abus adderal primarili studi 1-3 time per week befor recent soccer game took adderral night befor morn game deni short breath diaphoresi fever chill headach fatigu chang sleep chang vision/hear abdomin paun chang bowel urinari habit pmhx none rx use friend adderral fhx mom thyroid diseas dad recent heart attcak none immun date shx freshmen colleg endors 3-4 drink 3 night / week weekend deni tabacco endors tri marijuana sexual activ girlfriend x 1 year use condom,hpi 17yo present palpitation patient report 3-4 month intermittent episode heart beating/pounding chest 2 day ago soccer game episode time chest pressure felt going pas lose conciousness note patient endorses abusing adderall primarily study 1-3 time per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fever chill headache fatigue change sleep change vision/hearing abdominal paun change bowel urinary habit pmhx none rx us friend adderrall fhx mom thyroid disease dad recent heart attcak none immunization date shx freshman college endorses 3-4 drink 3 night / week weekend denies tabacco endorses trying marijuana sexually active girlfriend x 1 year us condom,142,case0


In [11]:
# Split X Y
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df.lemmatized)
y = df.case_num

In [12]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2, random_state=123)

In [14]:
X_train_view = X_train.toarray()


In [15]:
X_train_view

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Baseline

In [49]:
df.case_num.value_counts()

5    1800
8    1800
2    1700
9    1700
3    1600
0    1300
1    1300
6    1200
4    1000
7     900
Name: case_num, dtype: int64

Can set baseline to be case 5

In [61]:
# baseline
df[df['target'] == 'case5'].target.value_counts()/sum(df.target.value_counts())

case5    0.125874
Name: target, dtype: float64

Baseline is 12.58

In [70]:
#Create evaluation dataframe
train = pd.DataFrame(dict(actual=y_train))
train['baseline']=5

In [71]:
train.head()

,actual,baseline
1543,1,5
6354,4,5
12324,8,5
8503,5,5
1243,0,5


In [73]:
train.baseline = train.baseline.astype('category')

In [74]:
train.dtypes

actual      category
baseline    category
dtype: object

In [75]:
# Calculate baseline model performance
print('BASELINE PERFORMANCE')
print('---')
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.baseline)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.actual, train.baseline))
print('---')
print(classification_report(train.actual, train.baseline))


Accuracy: 12.59%
---
Confusion Matrix
baseline     5
actual        
0         1040
1         1040
2         1360
3         1280
4          800
5         1440
6          960
7          720
8         1440
9         1360
---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1040
           1       0.00      0.00      0.00      1040
           2       0.00      0.00      0.00      1360
           3       0.00      0.00      0.00      1280
           4       0.00      0.00      0.00       800
           5       0.13      1.00      0.22      1440
           6       0.00      0.00      0.00       960
           7       0.00      0.00      0.00       720
           8       0.00      0.00      0.00      1440
           9       0.00      0.00      0.00      1360

    accuracy                           0.13     11440
   macro avg       0.01      0.10      0.02     11440
weighted avg       0.02      0.13      0.03     11440



/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---

## Create Models

### Logistic Regression

In [76]:
# Make and fit the object
lm = LogisticRegression().fit(X_train, y_train)
# Use it to make predictions
train['lm_predicted'] = lm.predict(X_train)
# Asssess accuracy
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.lm_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.lm_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.lm_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual           0     1     2     3    4     5    6    7     8     9
lm_predicted                                                         
0             1040     0     0     0    0     0    0    0     0     0
1                0  1040     0     0    0     0    0    0     0     0
2                0     0  1360     0    0     0    0    0     0     0
3                0     0     0  1280    0     0    0    0     0     0
4                0     0     0     0  800     0    0    0     0     0
5                0     0     0     0    0  1440    0    0     0     0
6                0     0     0     0    0     0  960    0     0     0
7                0     0     0     0    0     0    0  720     0     0
8                0     0     0     0    0     0    0    0  1440     0
9                0     0     0     0    0     0    0    0     0  1360
---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1040
        

In [77]:
cross_val_score(lm, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

### Decision Tree

In [78]:
# Make and fit the object
dtc = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
# Use the object
train['dt_predicted'] = dtc.predict(X_train)
# Determine performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.dt_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.dt_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.dt_predicted))

Accuracy: 34.63%
---
Confusion Matrix
actual           0     1     2     3    4     5    6    7     8     9
dt_predicted                                                         
2                0     0   202     0    0     0    0    7     0     0
5             1040  1040  1158  1280  640  1440   22  713    47  1360
6                0     0     0     0    0     0  927    0     0     0
8                0     0     0     0  160     0   11    0  1393     0
---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1040
           1       0.00      0.00      0.00      1040
           2       0.97      0.15      0.26      1360
           3       0.00      0.00      0.00      1280
           4       0.00      0.00      0.00       800
           5       0.16      1.00      0.28      1440
           6       1.00      0.97      0.98       960
           7       0.00      0.00      0.00       720
           8       0.89      0.97      0.93      14

/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
cross_val_score(dtc, X_train, y_train, cv = 5)


array([0.33916084, 0.34702797, 0.34353147, 0.34746503, 0.3833042 ])

### Random Forest

In [80]:
# Make and fit object
rf = RandomForestClassifier(bootstrap = True, 
                            class_weight = None, 
                            criterion = 'gini',
                            min_samples_leaf = 3,
                            n_estimators = 100,
                            max_depth = 8, 
                            random_state = 123).fit(X_train, y_train)
# Use it to make predictions
train['rf_predicted'] = rf.predict(X_train)
# Assess performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.rf_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.rf_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.rf_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual           0     1     2     3    4     5    6    7     8     9
rf_predicted                                                         
0             1040     0     0     0    0     0    0    0     0     0
1                0  1040     0     0    0     0    0    0     0     0
2                0     0  1360     0    0     0    0    0     0     0
3                0     0     0  1280    0     0    0    0     0     0
4                0     0     0     0  800     0    0    0     0     0
5                0     0     0     0    0  1440    0    0     0     0
6                0     0     0     0    0     0  960    0     0     0
7                0     0     0     0    0     0    0  720     0     0
8                0     0     0     0    0     0    0    0  1440     0
9                0     0     0     0    0     0    0    0     0  1360
---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1040
        

In [81]:
cross_val_score(rf, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

### KNN

In [82]:
# Make and fit the object
knn = KNeighborsClassifier(n_neighbors = 4).fit(X_train, y_train)
# Use the object 
train['knn_predicted'] = knn.predict(X_train)
# Evaluate performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.knn_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.knn_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.knn_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual            0     1     2     3    4     5    6    7     8     9
knn_predicted                                                         
0              1040     0     0     0    0     0    0    0     0     0
1                 0  1040     0     0    0     0    0    0     0     0
2                 0     0  1360     0    0     0    0    0     0     0
3                 0     0     0  1280    0     0    0    0     0     0
4                 0     0     0     0  800     0    0    0     0     0
5                 0     0     0     0    0  1440    0    0     0     0
6                 0     0     0     0    0     0  960    0     0     0
7                 0     0     0     0    0     0    0  720     0     0
8                 0     0     0     0    0     0    0    0  1440     0
9                 0     0     0     0    0     0    0    0     0  1360
---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1

In [83]:
# 5-fold Cross Validation
cross_val_score(knn, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

### Support Vector Classifier

In [84]:
# Make and fit the object
svc = LinearSVC(random_state=0).fit(X_train, y_train)
# Use the object
train['svc_predicted'] = svc.predict(X_train)
# Evaluate model
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.svc_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.svc_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.svc_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual            0     1     2     3    4     5    6    7     8     9
svc_predicted                                                         
0              1040     0     0     0    0     0    0    0     0     0
1                 0  1040     0     0    0     0    0    0     0     0
2                 0     0  1360     0    0     0    0    0     0     0
3                 0     0     0  1280    0     0    0    0     0     0
4                 0     0     0     0  800     0    0    0     0     0
5                 0     0     0     0    0  1440    0    0     0     0
6                 0     0     0     0    0     0  960    0     0     0
7                 0     0     0     0    0     0    0  720     0     0
8                 0     0     0     0    0     0    0    0  1440     0
9                 0     0     0     0    0     0    0    0     0  1360
---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1

In [85]:
# 5-fold Cross Validation
cross_val_score(svc, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

---

## Test

Test Knn model

In [86]:
# Create testing dataframe
test = pd.DataFrame(dict(actual=y_test))
test['knn_predicted'] = knn.predict(X_test)
# Evaluate model
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.knn_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.knn_predicted, test.actual))
print('---')
print(classification_report(test.actual, test.knn_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual           0    1    2    3    4    5    6    7    8    9
knn_predicted                                                  
0              260    0    0    0    0    0    0    0    0    0
1                0  260    0    0    0    0    0    0    0    0
2                0    0  340    0    0    0    0    0    0    0
3                0    0    0  320    0    0    0    0    0    0
4                0    0    0    0  200    0    0    0    0    0
5                0    0    0    0    0  360    0    0    0    0
6                0    0    0    0    0    0  240    0    0    0
7                0    0    0    0    0    0    0  180    0    0
8                0    0    0    0    0    0    0    0  360    0
9                0    0    0    0    0    0    0    0    0  340
---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       260
           1       1.00      1.00      1.00       260
           2       1.00   

In [87]:
# Calculate percent improvement from baseline accuracy (47%)
pct_improvement = round(((100-12.56)/(12.56)*100), 2)
print(f'Our 100% accuracy represent a {pct_improvement}% improvement from baseline')

Our 100% accuracy represent a 696.18% improvement from baseline
